In [256]:
import pandas as pd
from pyvis.network import Network
import networkx as nx

# Load the csv file
csv_file = "data.csv"
df = pd.read_csv(csv_file)

# Replace NaN in columns with empty strings
df['notes'] = df['notes'].fillna("")
df['connection_labels'] = df['connection_labels'].fillna("")
df['connection_directions'] = df['connection_directions'].fillna("")

# Initialise the pyvis network
net = Network(notebook=False, directed=True, cdn_resources="in_line")

# Add nodes with colors for albums, artists, loved items, and "to_listen"
for _, row in df.iterrows():
    # Assign colors based on type and love
    if row['type'] == "album":
        if row['to_listen'] == 1:
            color = "AliceBlue"  # Pale, soft blue for "to_listen"
        else:
            color = "#6495ED" if not row['love'] == 1 else "#4263B4"  # DarkCornflowerBlue for loved albums
    elif row['type'] == "artist":
        color = "SandyBrown" if not row['love'] == 1 else "DarkOrange"
    else:
        color = "gray"  # Default color if type is not recognized

    # Add node
    net.add_node(
        row['item_id'],
        label=row['item_name'],
        title=row['notes'],
        color=color
    )

# Add directional edges with labels
for _, row in df.iterrows():
    if pd.notna(row['connections']):
        connections = row['connections'].split('|')
        labels = row['connection_labels'].split('|') if row['connection_labels'] else [None] * len(connections)
        directions = row['connection_directions'].split('|') if row['connection_directions'] else ["none"] * len(connections)

        for conn, label, direction in zip(connections, labels, directions):
            arrow_to = True if direction == "forward" else False
            arrow_from = True if direction == "backward" else False
            net.add_edge(
                row['item_id'], int(conn),
                title=label,
                arrowStrikethrough=False,
                arrows={'to': arrow_to, 'from': arrow_from}
            )

# Save the HTML output with dynamic dimensions for responsiveness
net.set_options('''
{
  "layout": {
    "improvedLayout": true
  },
  "interaction": {
    "zoomView": true,
    "dragView": true
  },
  "physics": {
    "stabilization": {
      "enabled": true
    }
  }
}
''')

output_file = "index.html"
net.save_graph(output_file)

# Add responsive design meta tag and styles to the HTML
legend_html = """
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        body, html {
            margin: 0;
            padding: 0;
            width: 100%;
            height: 100%;
            overflow: hidden;
        }
        #mynetwork {
            width: 100%;
            height: calc(100vh - 50px);
        }
        .legend {
            position: absolute;
            bottom: 0;
            left: 0;
            right: 0;
            background: white;
            padding: 10px;
            font-family: Arial, sans-serif;
            font-size: 14px;
            border-top: 1px solid black;
            text-align: center;
        }
    </style>
</head>
"""

legend_html += """
<div class="legend">
    <b>Legend:</b>
    <ul style="list-style: none; padding: 0; margin: 0; display: inline-block; text-align: left;">
        <li><span style="background-color: #6495ED; display: inline-block; width: 15px; height: 15px; margin-right: 10px; border-radius: 50%;"></span>Album</li>
        <li><span style="background-color: #4263B4; display: inline-block; width: 15px; height: 15px; margin-right: 10px; border-radius: 50%;"></span>Favourite album</li>
        <li><span style="background-color: AliceBlue; display: inline-block; width: 15px; height: 15px; margin-right: 10px; border-radius: 50%;"></span>Album to listen</li>
        <li><span style="background-color: SandyBrown; display: inline-block; width: 15px; height: 15px; margin-right: 10px; border-radius: 50%;"></span>Artist</li>
        <li><span style="background-color: DarkOrange; display: inline-block; width: 15px; height: 15px; margin-right: 10px; border-radius: 50%;"></span>Favourite artist</li>
    </ul>
</div>
"""

# Insert the legend and meta tag into the saved HTML file
with open(output_file, "r") as file:
    html_content = file.read()

html_content = html_content.replace("<head>", legend_html + "<head>")
html_content = html_content.replace("</body>", "</body>")

with open(output_file, "w") as file:
    file.write(html_content)

print(f"Interactive network with legend saved as {output_file}")


Interactive network with legend saved as index.html
